In [4]:
import urllib
from urllib import request
from bs4 import BeautifulSoup
import os
import requests
import argparse
import re
import time
import json
import math
import codecs
import urllib.parse as up

In [41]:
# Generate a concatenated tldr string of papers from a certain query. Also return the information pack of these papers.
def SSSQuery(query, num_item=50, offset=0, fos=None):
    # Send a Semantic API post to get the result paper list
    query = '+'.join(query.split())
    url = f'https://api.semanticscholar.org/graph/v1/paper/search?query={query}&offset={offset}&limit={num_item}&fields=fieldsOfStudy,abstract'
    paper_list = str(request.urlopen(url).read(), 'utf-8')
    paper_list = json.loads(paper_list)

    tldr_cat = ''
    info_pack = []
    # Loop through the paper list and get information pack for each paper
    for paper in paper_list['data']:
        paper_id = paper['paperId']
        # Each paper info pack consists title, abstract, authors, tldr, citation count, field of study
        paper_url = f'https://api.semanticscholar.org/graph/v1/paper/{paper_id}?fields=title,abstract,citations.authors,tldr,citationCount,fieldsOfStudy'
        paper_info = str(request.urlopen(paper_url).read(), 'utf-8')
        paper_info = json.loads(paper_info)
        # Filter out the papers out of field of study
        if fos is not None and fos not in paper_info['fieldsOfStudy']:
            continue
        info_pack.append(paper_info)
        tldr = paper_info['tldr']
        if tldr is not None:
            tldr_cat += ' '+tldr['text'].strip()
    tldr_cat = tldr_cat.strip()
    return tldr_cat, info_pack

In [40]:
tldr_cat, info_pack = SSSQuery('transformer', num_item=10, fos='Computer Science')
print([i['citationCount'] for i in info_pack])

[4100, 2250, 1413, 531, 491, 313, 146, 183, 184, 134]

In [39]:
tldr_cat

'This work introduces a new learnable module, the Spatial Transformer, which explicitly allows the spatial manipulation of data within the network, and can be inserted into existing convolutional architectures, giving neural networks the ability to actively spatially transform feature maps. This systematic study compares pre-training objectives, architectures, unlabeled datasets, transfer approaches, and other factors on dozens of language understanding tasks and achieves state-of-the-art results on many benchmarks covering summarization, question answering, text classification, and more. This work proposes a novel neural architecture Transformer-XL that enables learning dependency beyond a fixed length without disrupting temporal coherence, which consists of a segment-level recurrence mechanism and a novel positional encoding scheme. Following prior work on long-sequence transformers, the Longformer is evaluated on character-level language modeling and achieves state-of-the-art result

# Development Scratch

In [21]:
num_item = 30
offset = 0
query = 'hand'
query = '+'.join(query.split())
url = f'https://api.semanticscholar.org/graph/v1/paper/search?fos[0]=computer-science&query={query}&offset={offset}&limit={num_item}'
paper_list = str(request.urlopen(url).read(), 'utf-8')
paper_list = json.loads(paper_list)
# print(paper_list)

In [42]:
for paper in paper_list['data']:
    paper_id = paper['paperId']
    paper_url = f'https://api.semanticscholar.org/graph/v1/paper/{paper_id}?fields=citations.authors,tldr,citationCount,fieldsOfStudy'
    paper_info = str(request.urlopen(paper_url).read(), 'utf-8')
    paper_info = json.loads(paper_info)
    print(paper_info['citationCount'], paper_info['fieldsOfStudy'])